In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk.data
import nltk
import glob
import os
import re
import string
import pandas as pd
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import numpy as np
from sklearn import preprocessing
from numpy import dot
from numpy.linalg import norm
from pyemd import emd
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
meeting_data = pd.read_pickle("pickleddf.pkl")

In [7]:
meeting_data.head()

,date,text,vector_average
0,2015-03-26,Mayor pro tem I had a que stion for Mr Mckinn...,"[0.03370697, 0.021581726, 0.024202384, 0.09209..."
1,2015-03-26,There are 20 that are signed up for the next ...,"[0.02433229, 0.03419032, 0.00010202286, 0.1011..."
2,2015-03-26,Okay The other thing is Im going to do everyt...,"[0.041835412, 0.021017635, 0.021236643, 0.1323..."
3,2015-03-26,Thanks Hello My name is Jacquie benastante an...,"[0.012454927, 0.028385554, 0.027422577, 0.0879..."
4,2015-03-26,Good afternoon Three of us are here today and...,"[-0.018927153, 0.027210213, 0.046535354, 0.069..."


In [3]:
meeting_data["vector_average"][0].shape

(300,)

In [26]:
# mock user input
text = "bike"

# clean and tokenize user input
tokenized_input = nltk.word_tokenize(text)

tokenized_input

['bike']

In [24]:
# get vectors for input text
#filename = 'GoogleNews-vectors-negative300.bin'
emb_model = KeyedVectors.load_word2vec_format(filename, binary=True)
vectorized_input = emb_model[tokenized_input]

vectorized_input[0].shape
#vectorized_input

NameError: name 'tokenized_input' is not defined

In [199]:
# vectorized input size needs to be 1, 300
meeting_data['vector_average'][0].shape

(300,)

In [213]:
meeting_data.head()

,date,text,vector_average
0,2015-03-26,Mayor pro tem I had a que stion for Mr Mckinn...,"[0.03370697, 0.021581726, 0.024202384, 0.09209..."
1,2015-03-26,There are 20 that are signed up for the next ...,"[0.02433229, 0.03419032, 0.00010202286, 0.1011..."
2,2015-03-26,Okay The other thing is Im going to do everyt...,"[0.041835412, 0.021017635, 0.021236643, 0.1323..."
3,2015-03-26,Thanks Hello My name is Jacquie benastante an...,"[0.012454927, 0.028385554, 0.027422577, 0.0879..."
4,2015-03-26,Good afternoon Three of us are here today and...,"[-0.018927153, 0.027210213, 0.046535354, 0.069..."


In [214]:
# test function
def get_cosine(array1,array2):
  array1 = array1.reshape(1,-1)
  array2 = array2.reshape(1,-1)
  return cosine_similarity(array1,array2)

In [ ]:
# write in my placeholders
def get_cosine(array1,array2):
  array1 = array1.reshape(1,-1)
  array2 = array2.reshape(1,-1)
  return cosine_similarity(array1,array2)

In [165]:
# calculate cosine similarity
cos_sim_input_data = []
for i in meeting_data['corrected_vectors']:
    cos_sim = cosine_similarity(i, vectorized_input.transpose())
    cos_sim_input_data.append(cos_sim)
    #print(vectorized_input.transpose().shape)


In [225]:
cosine_sim[0,0] = get_cosine(meeting_data['vector_average'][0], vectorized_input[0])
cosine_sim

0.26909935

In [226]:
cosine_sim = get_cosine(meeting_data['vector_average'][0], vectorized_input[0])[0, 0]
cosine_sim

0.26909935

In [96]:
## Failed reshape attempts - keeping for reference
#numpy.reshape(meeting_data['vector_average'][0], 1, 300)
#meeting_data['vector_average'][0].newaxis()
#array.reshape(-1, 1)
#vectorized_input[0].reshape(-1, 1)
#meeting_data["corrected_vectors"][0].reshape(-1, 1)

#test.transpose().shape
#meeting_data['vector_average'][0].shape

In [145]:
#cosine_similarity(vectorized_input[0].reshape(-1, 1), meeting_data["corrected_vectors"][0].reshape(-1, 1))
cosine_similarity(vectorized_input.reshape(-1, 1), meeting_data["corrected_vectors"][0].reshape(-1, 1))

array([[-1., -1., -1., ...,  1., -1.,  1.],
       [-1., -1., -1., ...,  1., -1.,  1.],
       [-1., -1., -1., ...,  1., -1.,  1.],
       ...,
       [ 1.,  1.,  1., ..., -1.,  1., -1.],
       [-1., -1., -1., ...,  1., -1.,  1.],
       [-1., -1., -1., ...,  1., -1.,  1.]], dtype=float32)

In [168]:
meeting_data["cos_sim"] = cos_sim_input_data

In [200]:
vectors_1d = []
for nd_vec in meeting_data.vector_average:
    vectors_1d.append(nd_vec[:,np.newaxis])
    
meeting_data["corrected_vectors"] = vectors_1d

In [4]:
#filename = 'GoogleNews-vectors-negative300.bin'
#emb_model = KeyedVectors.load_word2vec_format(filename, binary=True)

def activistsdigest():
    # accept user input
    text = input("Write a topic of interest here: ")

    # clean and tokenize user input
    tokenized_input = nltk.word_tokenize(text)
    
    # get vectors for input text
    #filename = 'GoogleNews-vectors-negative300.bin'
    #emb_model = KeyedVectors.load_word2vec_format(filename, binary=True)
    vectorized_input = emb_model[tokenized_input]

    #def get_cosine(array1,array2):
    #  array1 = array1.reshape(1,-1)
    #  array2 = array2.reshape(1,-1)
    #  return cosine_similarity(array1,array2)
    
    cosine_list = []
    for vector in meeting_data['vector_average']:
        vector.reshape(1, -1)
        cosine_sim = cosine_similarity(vector.reshape(1, -1), vectorized_input[0].reshape(1, -1))[0, 0]
        #cosine_sim = get_cosine(meeting_data['vector_average'][0], vectorized_input[0])[0, 0]
        
        cosine_list.append(cosine_sim)
    
    # add input_veclist to df
    meeting_data['cosine_similarity'] = cosine_list

    # sort dataframe in ascending order by distance
    meeting_data1 = meeting_data.sort_values(by=['cosine_similarity'], ascending = False)

    # return df for top 10 rows
    return print(meeting_data1.head(10))

In [5]:
activistsdigest()

Write a topic of interest here:  veteran care


            date                                               text  \
1282  2015-09-17   Its a great network of veterans seasoned vete...   
6625  2017-11-09   Mr Mayor you made an error when you said were...   
9914  2019-08-22   Mayor adlerlast but not least Charles Bryant ...   
488   2015-05-21   I met Betty in the 80s as well waltzed into t...   
7640  2018-06-28   You have a chartered flag there You ready to ...   
3915  2016-11-10   Good afternoon Gus peña proud native e ast Au...   
8057  2018-08-30   I ask you as an advocate of victims of violen...   
8154  2018-11-29   Thank you mayor and council members of the pu...   
8478  2019-03-07   Good af ternoon city council members thank yo...   
8645  2019-04-11   Thank you The next speaker is Trevor baton an...   

                                         vector_average  cosine_similarity  
1282  [0.016679244, 0.11719236, 0.013261622, 0.09553...           0.463230  
6625  [0.036784925, 0.039827045, 0.034639057, 0.0441...         